# Trabalho 1 de IA - Alocação de Salas (Simples)
**Disciplina:** Inteligência Artificial 2025.2
**Professor:** Lucídio Cabral
**Tema:** 6. Alocação de salas (Timetabling Simplificado)

---
### 1. Definição do Problema e Dados
Configuração das disciplinas, salas e horários utilizando estruturas simples.

In [10]:
import heapq
import time
from copy import copy

# --- 1. CONFIGURAÇÃO DO AMBIENTE (VERSÃO DEFINITIVA) ---

# Apenas 3 horários (O segredo para não demorar 5 minutos)
HORARIOS = ["08:00", "10:00", "14:00"]

# 5 Salas (Bem restrito para forçar conflitos rápidos)
SALAS = [
    {'id': 'S_Grande', 'cap': 60}, # Só cabe as turmas gigantes aqui
    {'id': 'S_Media1', 'cap': 40},
    {'id': 'S_Media2', 'cap': 40},
    {'id': 'Lab_1',    'cap': 25},
    {'id': 'Lab_2',    'cap': 25}
]

# 9 Disciplinas
# Mantive a "pegadinha" (Pequenas antes das Grandes) para ele pensar um pouco,
# mas com menos opções de horário, ele resolve bem mais rápido.
DISCIPLINAS = [
    # --- Pequenas (Ocupam espaço indevido se ele não for esperto) ---
    {'id': 'D1', 'nome': 'Optativa A',   'prof': 'Prof_X', 'alunos': 15},
    {'id': 'D2', 'nome': 'Optativa B',   'prof': 'Prof_Y', 'alunos': 20},
    {'id': 'D3', 'nome': 'Seminários',   'prof': 'Prof_Z', 'alunos': 20},
    
    # --- Médias ---
    {'id': 'D4', 'nome': 'Redes',        'prof': 'Iguatemi', 'alunos': 30},
    {'id': 'D5', 'nome': 'Banco Dados',  'prof': 'Damires',  'alunos': 35},
    {'id': 'D6', 'nome': 'Eng. Soft',    'prof': 'Uyara',    'alunos': 35},

    # --- Grandes (As problemáticas ficam pro final) ---
    {'id': 'D7', 'nome': 'I.A.',         'prof': 'Lucidio',  'alunos': 55}, # Só cabe na S_Grande
    {'id': 'D8', 'nome': 'Cálculo',      'prof': 'Prof_W',   'alunos': 50}, 
    {'id': 'D9', 'nome': 'Algoritmos',   'prof': 'Lucidio',  'alunos': 45}  # Conflito Lucidio + S_Grande
]

print(f"Dados configurados: 3 Horários, 5 Salas, 9 Disciplinas.")

Dados configurados: 3 Horários, 5 Salas, 9 Disciplinas.


In [11]:
# --- 2. MODELAGEM FORMAL (ESTADOS E AÇÕES) ---

def estado_inicial():
    # O estado é uma tupla: (dicionario_alocacoes, lista_ids_disciplinas_restantes)
    # alocacoes: { 'id_disciplina': ('id_sala', 'horario') }
    ids_restantes = tuple([d['id'] for d in DISCIPLINAS])
    return ({}, ids_restantes)

def eh_objetivo(estado):
    _, restantes = estado
    return len(restantes) == 0

def obter_disciplina(id_disc):
    # Função auxiliar para pegar os dados da disciplina pelo ID
    return next(d for d in DISCIPLINAS if d['id'] == id_disc)

def movimento_valido(alocacoes_atuais, disciplina, sala, horario):
    # RESTRIÇÃO 1: Capacidade [cite: 112]
    if sala['cap'] < disciplina['alunos']:
        return False

    # Verifica conflitos com o que já está alocado
    for id_alocada, (id_sala_alocada, horario_alocado) in alocacoes_atuais.items():
        if horario_alocado == horario:
            # RESTRIÇÃO 2: Sala já ocupada
            if id_sala_alocada == sala['id']:
                return False
            
            # RESTRIÇÃO 3: Professor já ocupado (Conflito) [cite: 112]
            disc_alocada = obter_disciplina(id_alocada)
            if disc_alocada['prof'] == disciplina['prof']:
                return False
                
    return True

def gerar_sucessores(estado):
    alocacoes, restantes = estado
    sucessores = []
    
    if not restantes:
        return []
    
    # Pega a próxima disciplina da lista para tentar alocar
    id_prox = restantes[0]
    novas_restantes = restantes[1:] # Remove a disciplina da lista
    dados_disc = obter_disciplina(id_prox)
    
    # Tenta todas as combinações de sala e horário (Ações Possíveis) [cite: 115]
    for sala in SALAS:
        for horario in HORARIOS:
            if movimento_valido(alocacoes, dados_disc, sala, horario):
                # Cria nova alocação copiando a anterior
                novas_alocacoes = alocacoes.copy()
                novas_alocacoes[id_prox] = (sala['id'], horario)
                
                # Novo estado
                sucessores.append((novas_alocacoes, novas_restantes))
                
    return sucessores

In [12]:
# --- 3. ALGORITMOS DE BUSCA ---

# HEURÍSTICA: Número de disciplinas restantes [cite: 118]
# Quanto menos faltam, mais perto do fim.
def heuristica(estado):
    _, restantes = estado
    return len(restantes)

# --- ESTRATÉGIA 1: A* (Informada) [cite: 13] ---
def busca_a_star(inicio):
    # Fila de prioridade: (f_score, contador_desempate, estado)
    fronteira = []
    contador = 0 
    
    # g = 0 (custo atual), h = disciplinas restantes
    heapq.heappush(fronteira, (heuristica(inicio), contador, inicio))
    
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        _, _, estado_atual = heapq.heappop(fronteira)
        visitados += 1
        
        # Teste de Objetivo [cite: 10]
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        # Custo g é o número de alocações feitas (len da dict de alocacoes)
        alocacoes, _ = estado_atual
        g = len(alocacoes) + 1 
        
        for sucessor in gerar_sucessores(estado_atual):
            h = heuristica(sucessor)
            f = g + h
            contador += 1
            heapq.heappush(fronteira, (f, contador, sucessor))
            
    return None, visitados, time.time() - tempo_ini

# --- ESTRATÉGIA 2: Híbrida BFS + DFS (Sem Informação explícita) [cite: 14] ---
def busca_hibrida(inicio, limite_bfs=2):
    # Usa uma lista simples como deque
    fronteira = [inicio]
    visitados = 0
    tempo_ini = time.time()
    
    while fronteira:
        # HIBRIDIZAÇÃO[cite: 123]:
        # Se alocamos poucas coisas (profundidade < limite), usa BFS (tira do início 0)
        # Se já alocamos bastante, mergulha com DFS (tira do final -1)
        
        profundidade_atual = len(fronteira[0][0]) # tamanho do dict alocacoes do 1º da fila
        
        if profundidade_atual < limite_bfs:
            estado_atual = fronteira.pop(0) # BFS
        else:
            estado_atual = fronteira.pop()  # DFS
            
        visitados += 1
        
        if eh_objetivo(estado_atual):
            return estado_atual, visitados, time.time() - tempo_ini
            
        # Adiciona sucessores ao final
        for sucessor in gerar_sucessores(estado_atual):
            fronteira.append(sucessor)
            
    return None, visitados, time.time() - tempo_ini

In [13]:
# --- 4. EXPERIMENTOS E RESULTADOS ---

def imprimir_solucao(nome_algoritmo, resultado):
    estado, nos, tempo = resultado
    print(f"\n>>> {nome_algoritmo}")
    print(f"Tempo: {tempo:.4f}s | Nós gerados: {nos}")
    
    if estado:
        alocacoes, _ = estado
        print(f"{'DISCIPLINA':<12} | {'SALA':<5} | {'HORÁRIO'}")
        print("-" * 35)
        for d_id, (s_id, hor) in alocacoes.items():
            nome_disc = obter_disciplina(d_id)['nome']
            print(f"{nome_disc:<12} | {s_id:<5} | {hor}")
    else:
        print("Nenhuma solução encontrada.")

# Rodando...
inicio = estado_inicial()

# Executa A*
res_a = busca_a_star(inicio)
imprimir_solucao("A* (Heurística: nº disciplinas restantes)", res_a)

# Executa Híbrido
# limite_bfs=2 significa: tenta todas as combinações para as 2 primeiras disciplinas (BFS),
# depois tenta fechar a grade o mais rápido possível (DFS).
res_h = busca_hibrida(inicio, limite_bfs=2)
imprimir_solucao("Híbrido (BFS nas 2 primeiras fases -> DFS)", res_h)


>>> A* (Heurística: nº disciplinas restantes)
Tempo: 107.6856s | Nós gerados: 2716565
DISCIPLINA   | SALA  | HORÁRIO
-----------------------------------
Optativa A   | S_Media1 | 08:00
Optativa B   | S_Media1 | 10:00
Seminários   | S_Media1 | 14:00
Redes        | S_Media2 | 08:00
Banco Dados  | S_Media2 | 10:00
Eng. Soft    | S_Media2 | 14:00
I.A.         | S_Grande | 08:00
Cálculo      | S_Grande | 10:00
Algoritmos   | S_Grande | 14:00

>>> Híbrido (BFS nas 2 primeiras fases -> DFS)
Tempo: 0.0008s | Nós gerados: 24
DISCIPLINA   | SALA  | HORÁRIO
-----------------------------------
Optativa A   | Lab_2 | 14:00
Optativa B   | Lab_2 | 10:00
Seminários   | Lab_2 | 08:00
Redes        | S_Media2 | 14:00
Banco Dados  | S_Media2 | 10:00
Eng. Soft    | S_Media2 | 08:00
I.A.         | S_Grande | 14:00
Cálculo      | S_Grande | 10:00
Algoritmos   | S_Grande | 08:00


### 5. Análise dos Resultados

Conforme observado nos experimentos, houve uma diferença significativa de desempenho entre as abordagens devido à natureza das restrições do problema de Alocação de Salas (Timetabling).

1.  **Desempenho do A*:**
    * A heurística utilizada ("Número de disciplinas restantes") é admissível, mas permitiu que o algoritmo explorasse caminhos inviáveis (alocando turmas pequenas em salas grandes precocemente).
    * Isso causou um alto número de nós gerados e *backtracking* excessivo quando o cenário de teste foi desenhado para ser desafiador (poucas salas grandes disponíveis).

2.  **Desempenho da Estratégia Híbrida (BFS/DFS):**
    * A estratégia híbrida mostrou-se superior neste cenário específico.
    * Ao utilizar DFS (Busca em Profundidade) após o nível inicial, o algoritmo conseguiu encontrar uma solução válida rapidamente ao mergulhar em um caminho promissor, em vez de tentar exaustivamente todas as combinações laterais como o A* ou BFS puro fariam.

**Conclusão:** Para problemas de satisfação de restrições rígidas (CSP) como este, onde o objetivo é encontrar *qualquer* solução válida (não necessariamente a ótima de menor custo de movimento), estratégias baseadas em profundidade (DFS) tendem a ser mais eficientes em memória e tempo do que buscas em largura, a menos que uma heurística muito forte (como *Minimum Remaining Values*) seja aplicada.